In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [15]:
wine_data = pd.read_csv('red.csv', delimiter=';')

In [18]:
'''Train data generation'''
duplicate_free_data = wine_data.drop_duplicates()
train, test = train_test_split(duplicate_free_data, test_size=0.2, shuffle=True, random_state=None)
